In [1]:
import pandas as pd
from pymongo import MongoClient
import time


In [2]:
import numpy as np

In [18]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.feature import CountVectorizer, IDF, CountVectorizerModel
from pyspark.sql.functions import udf

In [22]:
import pyspark as ps
spark = (ps.sql.SparkSession.builder
        .master("local[4]")
        .appName("scrap")
        .getOrCreate()
        )
sc = spark.sparkContext

In [3]:
mc = MongoClient()
db  = mc['raw_restaurants']
biz = db['restaurants']

rv = db['reviews']
rv_s = db['reviews_scrap']

In [39]:
users = db['users']

In [4]:
rv_s.count_documents({}), rv.count_documents({}), users.count_documents({}), biz.count_documents({})

(12834, 25977, 16111, 8946)

In [25]:
#coun_m  = CountVectorizerModel.load('countvect')

In [101]:
kmean = KMeansModel.load('kmeans')

AnalysisException: 'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'

In [26]:
%%time
scrap = pd.DataFrame(list(rv_s.find({})))

CPU times: user 224 ms, sys: 143 ms, total: 367 ms
Wall time: 475 ms


In [37]:
scrap.dropna(axis = 0, inplace=True)

In [38]:
scrap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12731 entries, 86 to 12833
Data columns (total 9 columns):
_id         12731 non-null object
alias       12731 non-null object
biz_id      12731 non-null object
category    12731 non-null object
date        12731 non-null object
id          12731 non-null object
rating      12731 non-null object
text        12731 non-null object
user_id     12731 non-null object
dtypes: object(9)
memory usage: 994.6+ KB


In [28]:
scrap.columns

Index(['_id', 'alias', 'biz_id', 'category', 'date', 'id', 'rating', 'text',
       'user_id'],
      dtype='object')

In [30]:
len(scrap.biz_id.unique()), len(scrap.user_id.unique())

(119, 119)

In [40]:
unic_u = scrap.user_id.unique()


In [47]:
scrap.rating =scrap.rating.astype(float)

In [48]:
neg = scrap[scrap.rating<3].copy()

In [76]:
len(neg)

1766

In [80]:
unic_b = neg.alias.unique()
len(unic_b)

1686

In [85]:
b = neg.groupby('alias').count() 
norm_rests = b[b.user_id >1].index
len(norm_rests)

70

In [86]:
len(neg[~neg.alias.isin(norm_rests)])

1616

In [87]:
len(neg[neg.alias.isin(norm_rests)])

150

In [90]:
text_for_user = neg[~neg.alias.isin(norm_rests)][['text','user_id']]

In [96]:
text_for_user.to_csv('user_to_cluster.csv')

In [97]:
us = text_for_user.user_id.unique()

150

146